In [156]:
import wikipedia
import codecs
import collections
import sys
import string
import re

#### Скачивание статей из Википедии

In [4]:
def get_texts_for_lang(lang, n=10): # функция для скачивания статей из википедии
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

## Языки

In [5]:
langs = ('kk', 'uk', 'be', 'fr')

* Казахский (kk)
* Украинский (uk)
* Белорусский (be)
* Французский (fr)

#### Создание корпусов

In [6]:
wiki_texts = {}
for lang in langs:
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

kk 100


/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Skipping page Княжицька сільська рада
Skipping page Задорожна
Skipping page Фреден
Skipping page Демкович
Skipping page Сенан
Skipping page Ламбет (значення)
uk 94
Skipping page Монькі (значэнні)
Skipping page Уладычына
Skipping page Асавец
be 97
Skipping page Actif
Skipping page Mauvaise Fille (film, 1991)
Skipping page Pleurodema elegans
Skipping page Gold
Skipping page Hernie
Skipping page La Pucelle d'Orléans (Schiller)
Skipping page Reaper
fr 93


## Первый метод: частотные слова

#### Чистим текст от тегов

In [170]:
def remove_tags(text):
    no_tags_text = re.sub(r'<[^>]+>', ' ', text)
    no_space_sequences_text = re.sub('  +', ' ', no_tags_text)
    return no_space_sequences_text

#### Токенизация

In [171]:
# удаляем знаки препинания
def tokenize(text):
    punct_extended = string.punctuation + '«»—…“”'
    table = str.maketrans({ch: None for ch in punct_extended})
    return [word.translate(table) for word in text.split()]

#### Список частотных слов

In [172]:
def frequent_list(langs, wiki_texts):
    lang_freqs = {}
    for lang in wiki_texts:
        corpus = wiki_texts[lang]
        lang_freqs[lang] = collections.defaultdict(lambda: 0)
        for article in corpus:
            for word in tokenize(remove_tags(article.replace('\n', '').lower())):
                lang_freqs[lang][word] += 1
    return lang_freqs

#### Фильтрация повторяющихся токенов

In [173]:
def filter_tokens(d):
    result = {}
    toks = []
    for key, value in d.items():
        result[key] = {}
        toks = toks + list(value.keys())
    duplicates  = [item for item, count in collections.Counter(toks).items() if count > 1]
    for key2, value2 in d.items():
        for key3, value3 in value2.items():
            if key3 not in duplicates:
                result[key2].update({key3 : value3})
    return(result)

#### Список  самых частотных слов

In [174]:
def most_frequent(d, keys, num):
    most_freqs = {}
    for k in keys:
        most_freqs[k] = set(sorted(d[k], key=lambda w: d[k][w], reverse=True)[:num])
    return most_freqs

In [175]:
def main_freq(langs, wiki_texts):
    # составляем частотный список
    lang_freqs = frequent_list(langs, wiki_texts)
    # фильтруем токены
    lang_freqs_filtered = filter_tokens(lang_freqs)
    # достаём самые частотные
    most_freq = most_frequent(lang_freqs_filtered, langs, 300)
    return most_freq

#### Определение языка корпусным методом

In [176]:
def corpus_method(text, most_freq):
    result = {}
    for lang in most_freq:
        result[lang] = len([word for word in tokenize(remove_tags(text.replace('\n', '').lower())) if word in most_freq[lang]])
    return sorted(result, key=lambda w: result[w], reverse=True)[0]

# Второй метод: частотные символьные n-граммы

#### Преобразование строки в массив n-грамм заданной длины

In [177]:
from itertools import islice, tee

def make_ngrams(text):
    N = 3 # задаем длину n-граммы
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams = [''.join(x) for x in ngrams]
    return ngrams

#### Список частотных ngram

In [178]:
def frequent_ngram_list(langs, wiki_texts):
    lang_freqs = {}
    for lang in wiki_texts:
        corpus = wiki_texts[lang]
        lang_freqs[lang] = collections.defaultdict(lambda: 0)
        for article in corpus:
            for ngram in make_ngrams(remove_tags(article.replace('\n', '').lower())):
                lang_freqs[lang][ngram] += 1
    return lang_freqs

In [179]:
def main_freq_ngram(langs, wiki_texts):
    # составляем частотный список
    lang_freqs = frequent_ngram_list(langs, wiki_texts)
    # фильтруем ngramы
    lang_freqs_filtered = filter_tokens(lang_freqs)
    # достаём самые частотные ngramы
    most_freq = most_frequent(lang_freqs_filtered, langs, 300)
    return most_freq

#### Определение языка n-граммным методом

In [188]:
def ngram_method(text, most_freq):
    result = {}
    for lang in most_freq:
        result[lang] = len([ngram for ngram in make_ngrams(remove_tags(text.replace('\n', '').lower())) if ngram in most_freq[lang]])
    return sorted(result, key=lambda w: result[w], reverse=True)[0]

#### Определяем язык двумя методами

In [189]:
def predict_lang(filename, most_freq, most_freq_n):
    text = open(filename,'r',encoding='utf-8').read()
    print('Корпусный метод: ', corpus_method(text, most_freq))
    print('n-граммный метод: ', ngram_method(text, most_freq_n))

In [190]:
most_freq = main_freq(langs, wiki_texts)
most_freq_n = main_freq_ngram(langs, wiki_texts)

In [191]:
predict_lang('/Users/irene/Downloads/HW2/fr.txt', most_freq, most_freq_n)

Корпусный метод:  fr
n-граммный метод:  fr
